# Ionospheric correction by Klobuchar model

The ionospheric effect is a part of atmospheric effect. The electomagnetic wave moves in a sparse and partly ionised gas in the higher layer of the atmosphere, which is called iononosphere. The ionospheric effect propotionate the count of free electrons along the way of the wave.

The Klobuchar model estimates the ionospheric effect.

## Input data

 - The GPS time (in seconds from midnight)
 - The coordinates of the user (receiver) (longitude and latitude in WGS84)
 - The direction of the satellite from the receiver (azimuth and elevation angle)
 - The ionospheric coefficients (from the navigation message)
 
The Klobuchar model uses semicircles to measure the angles.

In [7]:
# functions from the math module
from math import pi, sin, cos, copysign

# input data - time
hours=13
minutes=42
seconds=11.21456
#input data - receiver pos
user_lat_deg=47.1888
user_lon_deg=18.4188
#input data - satellite directon from the receiver
elev_deg=78.2541
az_deg=191.1257
#input data - ionospheric coefficient
alfa0= 1.49E-8
alfa1= 2.24E-8
alfa2=-1.19E-7
alfa3=-1.19E-7
beta0= 1.17E+5
beta1= 1.80E+5
beta2=-1.31E+5
beta3=-4.95E+5

# calculate the angles in semicircles, and the time from midnight in seconds
t_gps=hours*3600+minutes*60+seconds
user_lat=rec_lat_deg/180
user_lon=rec_lon_deg/180
elev=elev_deg/180
az=az_deg/180


## The ionospheric point (IPP)

Distance between the IPP and the receiver (Earth-centran angle in semicircles):
$$\psi=\frac{0.0137}{E+0.11}-0.022$$

In [8]:
psi=(0.0137/(elev+0.11))-0.022

The position of the IPP:

$$\phi_{i}=\begin{cases}
\phi_{u}+\psi\cos A & \left|\phi_{u}+\psi\cos A\right|\leq0.416\\
+0.416 & \phi_{u}+\psi\cos A>+0.416\\
-0.416 & \phi_{u}+\psi\cos A<-0.416
\end{cases}$$

$$\lambda_{i}=\lambda_{u}+\frac{\psi\sin A}{\cos\phi_{i}}$$

In [9]:
ipp_lat=user_lat+psi*cos(az*pi)
if abs(ipp_lat)>0.416:
    ipp_lat=copysign(0.416, ipp_lat)
ipp_lon=user_lon+(psi*sin(az*pi))/cos(ipp_lat*pi)

Geomagnetic latitude of the IPP:
$$\phi_{m}=\phi_{i}+0.064\cos\left(\lambda_{i}-1.617\right)$$

In [10]:
ipp_gmlat=ipp_lat+0.064*cos(ipp_lon-1.617)

Local time in the IPP:
$$t=43200\lambda_{i}+t_{GPS}$$

In [11]:
t=43200*ipp_lon+t_gps

## The ionospheric effect

$$F=1.0+16.0\left(0.53-E\right)^{3}$$

$$PER=\begin{cases}
\sum_{n=0}^{3}\beta_{n}\phi_{m}^{n} & \sum_{n=0}^{3}\beta_{n}\phi_{m}^{n}\geq72000\\
72000 & \sum_{n=0}^{3}\beta_{n}\phi_{m}^{n}<72000
\end{cases}$$

$$x=\frac{2\pi\left(t-50400\right)}{PER}$$

$$AMP=\begin{cases}
\sum_{n=0}^{3}\alpha_{n}\phi_{m}^{n} & \sum_{n=0}^{3}\alpha_{n}\phi_{m}^{n}\geq0\\
0 & \sum_{n=0}^{3}\alpha_{n}\phi_{m}^{n}<0
\end{cases}$$

$$T_{iono}=\begin{cases}
F\cdot\left(5.0\cdot10^{-9}+AMP\cdot\left(1-\frac{x^{2}}{2}+\frac{x^{4}}{24}\right)\right) & \left|x\right|<1.57\\
F\cdot5.0\cdot10^{-9} & \left|x\right|\geq1.57
\end{cases}$$

In [12]:
f=1.0+16.0*(0.53-elev)**3
per=min(72000, beta0+beta1*ipp_gmlat+beta2*ipp_gmlat**2+beta3*ipp_gmlat**3)
x=(2*pi*(t-50400))/per
amp=min(0, alfa0+alfa1*ipp_gmlat+alfa2*ipp_gmlat**2+alfa3*ipp_gmlat**3)
if abs(x)<1.57:
    t_iono=f*(5E-9+amp*(1-(x**2)/2+(x**4)/24))
else:
    t_iono=f*5E-9

t_iono

5.06914381389651e-09

in meters:

In [13]:
t_iono*299792458.0

1.5196910839235294

## References

 - Bernhard Hofmann-Wellenhof, Herbert Lichtenegger, Elmar Wasle: GNSS — Global Navigation Satellite Systems, Springer-Verlag Wien, 2008. Subsection 5.3.2
 - IS-GPS-200K
 - https://gssc.esa.int/navipedia/index.php/Klobuchar_Ionospheric_Model